# DCGAN (Deep Convolutional Generative Adversarial Network)

- Published in 2015 by Radford et al.
- Similar to the original GAN, but uses Deep Convolutional Networks instead of fully-connected
- This implies that the DCGAN is more suitable to work with images and videos
- The generator receive as input a random noise and returns a **fake** image
- The discriminator receives as input real and fake images and returns a value between **0** and **1** to indicates how much true is the image
- Uses **binary cross-entropy** as loss function
- The generator don't see the real images, just learns by the discriminator feedback

# Importing Libraries